# Missing Data Imputation
**Course**: HUDK 4051  

**Author**: Zecheng Chang

**Assignment**: ICE5  

**Objectives**:

At the end of this ICE, you will be able to:

* simulate MCAR data or MAR data
* implement common missing data imputation methods
* (optional) compare the performance of missing data imputation methods on an analysis you have done before.  

Missing data is very common in any work related to data analysis. Oftentimes, analyses will suffer from the problem that some observations/entries in the dataset are complete. In this ICE, we will explore a few common data imputation methods that deals with missing data.

## Missing data mechanism
There are 3 general mechanisms of missing data:  
* MCAR: Missing complete at random - Whether or not $Y_{i}$ is missing is independent of both $Y_{i}$ and any observed $X_{i}$
* MAR: Missing at Random - Whether or not $Y_{i}$ is missing is independent of $Y_{i}$ given observed covariates $X_{i}$
* NMAR (or sometimes MNAR): Not missing at random/Missing not at random - This is the most generic mechanism, where the cause of the missingness cannot be controlled. Whether or not $Y_{i}$ is missing is not independent even after controlling the observed covariates $X_{i}$


To illustrate, we are going to use the iris dataset, perhaps one of the best known dataset in data science. You can check out some more details about this dataset on UCI repo: https://archive.ics.uci.edu/ml/datasets/iris

In [18]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.impute import KNNImputer

In [3]:
irisRaw = datasets.load_iris()

# Convert it to a pandas dataframe for better visualization
iris = pd.DataFrame(data=irisRaw.data, columns=irisRaw.feature_names)
iris.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [4]:
iris.tail(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3
149,5.9,3.0,5.1,1.8


In [5]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB


In [7]:
iris.isnull().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
dtype: int64

## MCAR

MCAR is the easiest
Let make up some missing values randomly

In [6]:
iris_MCAR = iris.copy()

# set the missingness rate at 30%
missing = 0.3
iris_MCAR.loc[iris_MCAR.sample(frac=missing).index, 'sepal length (cm)'] = np.nan

iris_MCAR

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,NaN,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,NaN,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [8]:
# Alternatively, iterate through all columns and introduce 20% missing to all variables.
for col in iris_MCAR.columns:
    iris_MCAR.loc[iris_MCAR.sample(frac=missing).index, col] = np.nan


## MAR

Another easy way to generate an MAR data is to use the weights parameter in pd.sample().

In [10]:
iris_MAR = iris.copy()

# Set the missingness rate at 30%
missing = 0.3

# Only introduce missing rate to the `sepal length (cm)`.
iris_MAR.loc[iris_MAR.sample(frac=missing, weights = 'sepal width (cm)').index, 'sepal length (cm)'] = np.nan

iris_MAR

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


## Deletion Methods

### Listwise deletion

It produces unbiased estimation only when the data is MCAR. However, the reduced sample size leads to reduced power. In python, we can use dropna() to perform a listwise deletion. You can see here, as we introduced 30% of missing into the dataset, the sample size is now 105.

In [11]:
iris_MCAR_listwise = iris_MCAR.copy()
iris_MCAR_listwise.dropna()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
1,4.9,3.0,1.4,0.2
9,4.9,3.1,1.5,0.1
23,5.1,3.3,1.7,0.5
24,4.8,3.4,1.9,0.2
26,5.0,3.4,1.6,0.4
29,4.7,3.2,1.6,0.2
38,4.4,3.0,1.3,0.2
45,4.8,3.0,1.4,0.3
50,7.0,3.2,4.7,1.4
61,5.9,3.0,4.2,1.5


### Pairwise deletion

In [13]:
iris_MCAR_pairwise = iris_MCAR.copy()
iris_MAR_pairwise = iris_MAR.copy()

print("Mean estimation in MCAR data with pairwise deletion",
      iris_MCAR_pairwise['sepal length (cm)'].mean(), 
     "\nSD estimation in MCAR data with pairwise deletion",
      iris_MCAR_pairwise['sepal length (cm)'].std())


print("Mean estimation in MAR data with pairwise deletion",
      iris_MAR_pairwise['sepal length (cm)'].mean(), 
     "\nSD estimation in MAR data with pairwise deletion",
      iris_MAR_pairwise['sepal length (cm)'].std())


print("Mean estimation in the original data",
      iris['sepal length (cm)'].mean(),
      "\nSD estimation in the original data",
      iris['sepal length (cm)'].std())

Mean estimation in MCAR data with pairwise deletion 5.857894736842104 
SD estimation in MCAR data with pairwise deletion 0.8157226910978569
Mean estimation in MAR data with pairwise deletion 5.875238095238095 
SD estimation in MAR data with pairwise deletion 0.7785317730264588
Mean estimation in the original data 5.843333333333335 
SD estimation in the original data 0.8280661279778629


### Single Imputation

In [15]:
iris_MAR_mean = iris_MAR.copy()
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(iris_MAR)
print(imp.transform(iris_MAR))

[[5.1       3.5       1.4       0.2      ]
 [4.9       3.        1.4       0.2      ]
 [4.7       3.2       1.3       0.2      ]
 [4.6       3.1       1.5       0.2      ]
 [5.        3.6       1.4       0.2      ]
 [5.4       3.9       1.7       0.4      ]
 [5.8752381 3.4       1.4       0.3      ]
 [5.        3.4       1.5       0.2      ]
 [4.4       2.9       1.4       0.2      ]
 [5.8752381 3.1       1.5       0.1      ]
 [5.4       3.7       1.5       0.2      ]
 [4.8       3.4       1.6       0.2      ]
 [4.8       3.        1.4       0.1      ]
 [5.8752381 3.        1.1       0.1      ]
 [5.8       4.        1.2       0.2      ]
 [5.8752381 4.4       1.5       0.4      ]
 [5.8752381 3.9       1.3       0.4      ]
 [5.8752381 3.5       1.4       0.3      ]
 [5.7       3.8       1.7       0.3      ]
 [5.1       3.8       1.5       0.3      ]
 [5.8752381 3.4       1.7       0.2      ]
 [5.1       3.7       1.5       0.4      ]
 [5.8752381 3.6       1.        0.2      ]
 [5.1      

### Regression Imputation/Conditional Mean Imputation

In [17]:
iris_MAR_regression = iris_MAR.copy()
iris_MAR_regression_model = iris_MAR_regression.copy().dropna()

model = linear_model.LinearRegression()
Xs = iris_MAR_regression_model['sepal width (cm)'].values.reshape(-1, 1)
ys = iris_MAR_regression_model['sepal length (cm)'].values.reshape(-1, 1)
model.fit(X = Xs, y = ys)

null_index = iris_MAR_regression['sepal length (cm)'].isnull()

na_result = model.predict(iris_MAR_regression[null_index]['sepal width (cm)'].values.reshape(-1, 1))


iris_MAR_regression.loc[iris_MAR_regression['sepal length (cm)'].isnull(), 'sepal length (cm)'] = na_result.reshape(len(na_result),)

iris_MAR_regression

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


### Nearest neighbours or hot-deck imputation

In [19]:
iris_MAR_knn = iris_MAR.copy()
knn_imputer = KNNImputer(n_neighbors=2, weights="uniform")
knn_imputer.fit_transform(iris_MAR_knn)

array([[5.1 , 3.5 , 1.4 , 0.2 ],
       [4.9 , 3.  , 1.4 , 0.2 ],
       [4.7 , 3.2 , 1.3 , 0.2 ],
       [4.6 , 3.1 , 1.5 , 0.2 ],
       [5.  , 3.6 , 1.4 , 0.2 ],
       [5.4 , 3.9 , 1.7 , 0.4 ],
       [5.15, 3.4 , 1.4 , 0.3 ],
       [5.  , 3.4 , 1.5 , 0.2 ],
       [4.4 , 2.9 , 1.4 , 0.2 ],
       [4.75, 3.1 , 1.5 , 0.1 ],
       [5.4 , 3.7 , 1.5 , 0.2 ],
       [4.8 , 3.4 , 1.6 , 0.2 ],
       [4.8 , 3.  , 1.4 , 0.1 ],
       [4.55, 3.  , 1.1 , 0.1 ],
       [5.8 , 4.  , 1.2 , 0.2 ],
       [5.45, 4.4 , 1.5 , 0.4 ],
       [5.45, 3.9 , 1.3 , 0.4 ],
       [5.05, 3.5 , 1.4 , 0.3 ],
       [5.7 , 3.8 , 1.7 , 0.3 ],
       [5.1 , 3.8 , 1.5 , 0.3 ],
       [4.8 , 3.4 , 1.7 , 0.2 ],
       [5.1 , 3.7 , 1.5 , 0.4 ],
       [5.25, 3.6 , 1.  , 0.2 ],
       [5.1 , 3.3 , 1.7 , 0.5 ],
       [4.8 , 3.4 , 1.9 , 0.2 ],
       [5.  , 3.  , 1.6 , 0.2 ],
       [5.  , 3.4 , 1.6 , 0.4 ],
       [5.05, 3.5 , 1.5 , 0.2 ],
       [5.2 , 3.4 , 1.4 , 0.2 ],
       [4.7 , 3.2 , 1.6 , 0.2 ],
       [4.